# Examen Final - Recuperación de Información

In [ ]:
%pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 65.6 MB/s eta 0:00:00


In [ ]:
!pip install openai

In [ ]:
!pip install punctuation

ERROR: Could not find a version that satisfies the requirement punctuation (from versions: none)
ERROR: No matching distribution found for punctuation


In [ ]:
!pip install -q google-generativeai

In [ ]:
!pip install rank_bm25

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.4 MB/s eta 0:00:00


In [ ]:
# Librerias
import kagglehub
import pandas as pd
import json
import re
import os
import nltk
from tqdm import tqdm
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import google.generativeai as genai
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
from rank_bm25 import BM25Okapi
import faiss
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Descargar dataset (subset 1%)
path = kagglehub.dataset_download("Cornell-University/arxiv")
print("Ruta de los archivos del dataset:", path)

# Cargar el archivo JSONL
file_path = path + "/arxiv-metadata-oai-snapshot.json"
subset_size = 28054

data = []
with open(file_path, 'r') as f:
    for i, line in tqdm(enumerate(f), total=subset_size):
        if i >= subset_size:
            break
        data.append(json.loads(line))

# Crear DataFrame
df = pd.DataFrame(data)
df = df[["id", "title", "abstract"]]
df


Ruta de los archivos del dataset: /kaggle/input/arxiv


100%|██████████| 28054/28054 [00:01<00:00, 14315.59it/s]


,id,title,abstract
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-..."
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...
...,...,...,...
28049,0710.1102,Multiplicative Conservation of Baryon Number a...,In the canonical seesaw mechanism of neutrin...
28050,0710.1103,The Newton polygon of a rational plane curve,The Newton polygon of the implicit equation ...
28051,0710.1104,Smooth Volume Rigidity for Manifolds with Nega...,We establish conditions for a continuous map...
28052,0710.1105,Fractal and Multifractal Scaling of Electrical...,This article is a mini-review about electric...


## Preprocesamiento de texto

In [ ]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # eliminar puntuación
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in ENGLISH_STOP_WORDS]
    return " ".join(tokens)

tqdm.pandas()
df["text"] = (df["title"] + " " + df["abstract"]).progress_apply(preprocess)
df


100%|██████████| 28054/28054 [00:22<00:00, 1226.74it/s]


,id,title,abstract,text
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,calculation prompt diphoton production cross s...
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",sparsitycertifying graph decompositions new al...
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,evolution earthmoon based dark matter field fl...
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,determinant stirling cycle numbers counts unla...
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,dyadic lambda_alpha lambda_alpha paper compute...
...,...,...,...,...
28049,0710.1102,Multiplicative Conservation of Baryon Number a...,In the canonical seesaw mechanism of neutrin...,multiplicative conservation baryon number bary...
28050,0710.1103,The Newton polygon of a rational plane curve,The Newton polygon of the implicit equation ...,newton polygon rational plane curve newton pol...
28051,0710.1104,Smooth Volume Rigidity for Manifolds with Nega...,We establish conditions for a continuous map...,smooth volume rigidity manifolds negatively cu...
28052,0710.1105,Fractal and Multifractal Scaling of Electrical...,This article is a mini-review about electric...,fractal multifractal scaling electrical conduc...


##  Indexación

In [ ]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df["text"])

# BM25
bm25_tokenized = df["text"].apply(lambda x: x.split())
bm25_model = BM25Okapi(bm25_tokenized.tolist())


## Funciones de Búsqueda

In [ ]:
def search_tfidf(query, top_k=10):
    query_vec = tfidf_vectorizer.transform([preprocess(query)])
    scores = (tfidf_matrix @ query_vec.T).toarray().ravel()
    top_indices = scores.argsort()[-top_k:][::-1]
    return df.iloc[top_indices][["id", "title", "abstract"]]

def search_bm25(query, top_k=10):
    tokenized_query = preprocess(query).split()
    scores = bm25_model.get_scores(tokenized_query)
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
    return df.iloc[top_indices][["id", "title", "abstract"]]


## Embeddings + FAISS

In [ ]:
# Generar embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(df["text"].tolist(), show_progress_bar=True)

# Indexar con FAISS
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

In [ ]:
def search_faiss(query, top_k=10):
    query_vec = model.encode([preprocess(query)])
    D, I = index.search(np.array(query_vec), top_k)
    return df.iloc[I[0]][["id", "title", "abstract"]]

In [ ]:
def _format_results(indices, scores, top_k):
    # Ordenar por score descendente
    order = np.argsort(scores)[::-1][:top_k]
    indices = indices[order]
    scores = scores[order]

    rows = []
    for rank, (i, s) in enumerate(zip(indices, scores), start=1):
        abs_text = df.iloc[i]["abstract"] or ""
        snippet = abs_text[:300].replace("\n", " ") + ("..." if len(abs_text) > 300 else "")
        rows.append({
            "rank": rank,
            "id": df.iloc[i]["id"],
            "title": df.iloc[i]["title"],
            "snippet": snippet,
            "score": float(s)
        })
    return pd.DataFrame(rows, columns=["rank", "id", "title", "snippet", "score"])

## Módulo RAG

In [ ]:
from openai import OpenAI

# Crear cliente con tu API key de OpenAI
client = OpenAI(api_key="")

In [ ]:
def generate_rag_openai_with_context(query, top_k=3):
    # Recuperar documentos con cada modelo
    tfidf_top = search_tfidf(query, top_k=top_k)
    bm25_top = search_bm25(query, top_k=top_k)
    faiss_top = search_faiss(query, top_k=top_k)

    # Combinar contexto
    context_sections = []
    for df_result, name in [(tfidf_top, "TF-IDF"), (bm25_top, "BM25"), (faiss_top, "FAISS")]:
        for _, row in df_result.iterrows():
            abstract = (row['abstract'] or "")[:1000]
            section = f"[{name}] {row['title']}\n{abstract}"
            context_sections.append(section)

    context = "\n\n".join(context_sections)

    # Armar prompt en español (estilo RAG)
    prompt = f"""Eres una aplicación de tipo Retrieval-Augmented Generation (RAG) que siempre responde en español.
Tu tarea es ayudar al usuario a obtener información científica a partir del contexto proporcionado.
La respuesta debe explicar por qué los artículos son relevantes y dar un resumen del tema consultado.
Si la información no se encuentra en el contexto, responde con: "Lo siento, no tengo suficiente información para responder a esa pregunta."

Contexto:
{context}

Pregunta del usuario:
{query}
"""

    # Generar respuesta con GPT-4.1
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",  # puedes usar "gpt-4" si no tienes acceso a preview
        messages=[
            {"role": "system", "content": "Eres un asistente experto en recuperación de información científica."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )

    # Mostrar respuesta
    answer = response.choices[0].message.content
    print(f"\nConsulta: {query}")
    print(f"\nRespuesta:\n{answer}")
    return answer


In [ ]:
# Ejecutar la función para una consulta
generate_rag_openai_with_context("quantum chromodynamics", top_k=3)


Consulta: quantum chromodynamics

Respuesta:
Quantum Chromodynamics (QCD) es la teoría que describe la fuerza fuerte, una de las cuatro fuerzas fundamentales de la naturaleza. Esta fuerza es la responsable de mantener unidos a los quarks y gluones dentro de los hadrones, como los protones y neutrones. QCD es una parte esencial del Modelo Estándar de la física de partículas y se basa en la teoría de gauge con el grupo de simetría SU(3).

Los artículos proporcionados en el contexto abordan diferentes aspectos de la QCD y su aplicación en otros campos de la física:

1. "Monte Carlo Methods in Quantum Field Theory": Este artículo discute la aplicación de métodos de integración de Monte Carlo en la teoría de campos cuánticos, con un enfoque particular en la QCD. Los métodos de Monte Carlo son técnicas computacionales que permiten la evaluación de integrales complejas y son esenciales para realizar cálculos en QCD, especialmente en el estudio de la cromodinámica cuántica en retículos (latti

'Quantum Chromodynamics (QCD) es la teoría que describe la fuerza fuerte, una de las cuatro fuerzas fundamentales de la naturaleza. Esta fuerza es la responsable de mantener unidos a los quarks y gluones dentro de los hadrones, como los protones y neutrones. QCD es una parte esencial del Modelo Estándar de la física de partículas y se basa en la teoría de gauge con el grupo de simetría SU(3).\n\nLos artículos proporcionados en el contexto abordan diferentes aspectos de la QCD y su aplicación en otros campos de la física:\n\n1. "Monte Carlo Methods in Quantum Field Theory": Este artículo discute la aplicación de métodos de integración de Monte Carlo en la teoría de campos cuánticos, con un enfoque particular en la QCD. Los métodos de Monte Carlo son técnicas computacionales que permiten la evaluación de integrales complejas y son esenciales para realizar cálculos en QCD, especialmente en el estudio de la cromodinámica cuántica en retículos (lattice QCD).\n\n2. "Two interacting GL-equati

## Evaluación

In [ ]:
def compare_models(query, top_k=10, use_openai=True):
    print("="*120)
    print(f"Comparando modelos para la consulta: \"{query}\"\n")

    # Recuperar resultados
    tfidf_results = search_tfidf(query, top_k)
    bm25_results = search_bm25(query, top_k)
    faiss_results = search_faiss(query, top_k)

    # Obtener sets de IDs
    tfidf_ids = set(tfidf_results["id"])
    bm25_ids  = set(bm25_results["id"])
    faiss_ids = set(faiss_results["id"])

    # Evaluar intersecciones
    inter_all   = tfidf_ids & bm25_ids & faiss_ids
    inter_tf_bm = tfidf_ids & bm25_ids
    inter_tf_fs = tfidf_ids & faiss_ids
    inter_bm_fs = bm25_ids  & faiss_ids

    print(f"Coincidencias entre los 3 modelos: {len(inter_all)}")
    print(f"Coincidencias TF-IDF ∩ BM25:       {len(inter_tf_bm)}")
    print(f"Coincidencias TF-IDF ∩ FAISS:      {len(inter_tf_fs)}")
    print(f"Coincidencias BM25 ∩ FAISS:        {len(inter_bm_fs)}\n")

    if len(inter_all) > 0:
        print("Documentos comunes en los 3 modelos:")
        for i in inter_all:
            title = df[df["id"] == i]["title"].values[0]
            print(f"- {i}: {title}")

    print("\nEjecutando módulo RAG con contexto combinado...\n")

    # Llamar al RAG correspondiente
    if use_openai:
        response = generate_rag_openai_with_context(query, top_k=top_k)
    else:
        response = generate_rag_openai_with_context(query, top_k=top_k)

    print("\nRespuesta generada por RAG:\n")
    print(response)

    return {
        "query": query,
        "tfidf_ids": tfidf_ids,
        "bm25_ids": bm25_ids,
        "faiss_ids": faiss_ids,
        "inter_all": inter_all,
        "rag_response": response
    }


In [ ]:
compare_models("quantum chromodynamics", top_k=10, use_openai=True)

Comparando modelos para la consulta: "quantum chromodynamics"

Coincidencias entre los 3 modelos: 3
Coincidencias TF-IDF ∩ BM25:       8
Coincidencias TF-IDF ∩ FAISS:      3
Coincidencias BM25 ∩ FAISS:        3

Documentos comunes en los 3 modelos:
- 0705.4356: Monte Carlo Methods in Quantum Field Theory
- 0707.1065: Connection between some nonperturbative approaches in QCD
- 0705.3170: Two interacting GL-equations in High-T$_c$ superconductivity and quantum
  chromodynamics

Ejecutando módulo RAG con contexto combinado...


Consulta: quantum chromodynamics

Respuesta:
Los artículos proporcionados en el contexto abordan diversos aspectos de la Cromodinámica Cuántica (QCD, por sus siglas en inglés), que es la teoría que describe la interacción fuerte, una de las cuatro fuerzas fundamentales de la naturaleza. Esta interacción es responsable de la unión de quarks y gluones en partículas compuestas como protones y neutrones. A continuación, se presenta un resumen de los temas relevantes de

{'query': 'quantum chromodynamics',
 'tfidf_ids': {'0704.1724',
  '0704.2247',
  '0705.3170',
  '0705.4356',
  '0706.2623',
  '0707.0502',
  '0707.1065',
  '0707.3996',
  '0708.0047',
  '0708.2072'},
 'bm25_ids': {'0704.1724',
  '0705.3170',
  '0705.4046',
  '0705.4356',
  '0707.0502',
  '0707.1009',
  '0707.1065',
  '0707.3996',
  '0708.0047',
  '0708.2072'},
 'faiss_ids': {'0704.3214',
  '0705.2951',
  '0705.2974',
  '0705.3170',
  '0705.4356',
  '0706.4030',
  '0707.1065',
  '0707.3539',
  '0707.3697',
  '0708.0012'},
 'inter_all': {'0705.3170', '0705.4356', '0707.1065'},
 'rag_response': 'Los artículos proporcionados en el contexto abordan diversos aspectos de la Cromodinámica Cuántica (QCD, por sus siglas en inglés), que es la teoría que describe la interacción fuerte, una de las cuatro fuerzas fundamentales de la naturaleza. Esta interacción es responsable de la unión de quarks y gluones en partículas compuestas como protones y neutrones. A continuación, se presenta un resumen de